## OR Tools Scheduling


In [2]:
from ortools.sat.python import cp_model

### Simple Test


In [13]:
model = cp_model.CpModel()
x1 = model.new_int_var(0, 5, "x1")
x2 = model.new_int_var(0, 6, "x2")

model.maximize(0)

# Create the solver
solver = cp_model.CpSolver()

# Solve the model
status = solver.Solve(model)

print(solver.objective_value)
print(solver.value(x1))
print(solver.value(x2))

0.0
0
0


### Week Problem

In [44]:
# Define the problem
model = cp_model.CpModel()
topics_per_day_target = [2, 2, 2]
n_days = len(topics_per_day_target)
topics_to_days = {
    "Maths": model.new_int_var(0, n_days - 1, "Maths"),
    "Physics": model.new_int_var(0, n_days - 1, "Physics"),
    "Chemistry": model.new_int_var(0, n_days - 1, "Chemistry"),
    "Biology": model.new_int_var(0, n_days - 1, "Biology"),
    "English": model.new_int_var(0, n_days - 1, "English"),
    "Further Maths": model.new_int_var(0, n_days - 1, "Further Maths"),
}

# Adding indicators to check if a topic is on a specific day
indicators = [
    {
        topic_title: model.new_bool_var(f"{topic_title}_on_day_{day}")
        for topic_title in topics_to_days.keys()
    }
    for day in range(len(topics_per_day_target))
]

for day_num in range(len(indicators)):
    for topic_title, indicator in indicators[day_num].items():
        model.add(topics_to_days[topic_title] == day_num).only_enforce_if(indicator)
        model.add(topics_to_days[topic_title] != day_num).only_enforce_if(indicator.Not())

# Constraining such that the number of topics on each day matches the target
for day_indicators, n_topics in zip(indicators, topics_per_day_target):
    model.add(sum(day_indicators.values()) == n_topics)

# Define binary indicator variables
model.maximize(0)  # Dummy constraint

# Constraining the order of topics
model.add(topics_to_days["Maths"] <= topics_to_days["Physics"])
model.add(topics_to_days["Physics"] <= topics_to_days["Chemistry"])
model.add(topics_to_days["Chemistry"] <= topics_to_days["Biology"])
model.add(topics_to_days["Biology"] <= topics_to_days["English"])
model.add(topics_to_days["English"] <= topics_to_days["Further Maths"])


solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solution found:")
    for topic, var in topics_to_days.items():
        print(f"{topic} is scheduled on day {solver.value(var)}")

    print("")
    for day_num in range(len(indicators)):
        for topic_title, indicator in indicators[day_num].items():
            print(f"{indicator}: {solver.value(indicator)}")
else:
    print("No solution found.")

Solution found:
Maths is scheduled on day 0
Physics is scheduled on day 0
Chemistry is scheduled on day 1
Biology is scheduled on day 1
English is scheduled on day 2
Further Maths is scheduled on day 2

Maths_on_day_0: 1
Physics_on_day_0: 1
Chemistry_on_day_0: 0
Biology_on_day_0: 0
English_on_day_0: 0
Further Maths_on_day_0: 0
Maths_on_day_1: 0
Physics_on_day_1: 0
Chemistry_on_day_1: 1
Biology_on_day_1: 1
English_on_day_1: 0
Further Maths_on_day_1: 0
Maths_on_day_2: 0
Physics_on_day_2: 0
Chemistry_on_day_2: 0
Biology_on_day_2: 0
English_on_day_2: 1
Further Maths_on_day_2: 1
